In [1]:
import string

In [2]:
stopwords = nltk.corpus.stopwords.words('english')

In [3]:
ps = nltk.PorterStemmer()

In [4]:
data = pd.read_csv('data/SMSSpamCollection.csv')
data = data[['label','text']]
data.head()
data['text'] = data['text'].apply(str)
# data.dropna(axis=0, inplace=True)


In [5]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

In [6]:


data['body_len'] = data['text'].apply(lambda x: len(x) - x.count(" "))


In [7]:
data['punct%'] = data['text'].apply(lambda x: round(count_punct(x) / len(x), 3) * 100)

In [8]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens =  re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [9]:
# split
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data[['text', 'body_len', 'punct%']], data['label'], test_size=0.2)

In [12]:
# vectorize
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['text'])

In [13]:
tfidf_train = tfidf_vect_fit.transform(X_train['text'])
tfidf_test = tfidf_vect_fit.transform(X_test['text'])

In [14]:
X_train_vect = pd.concat([X_train[['body_len', 'punct%']].reset_index(drop=True), pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['body_len', 'punct%']].reset_index(drop=True), pd.DataFrame(tfidf_test.toarray())], axis=1)

In [15]:
X_train_vect.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,6040,6041,6042,6043,6044,6045,6046,6047,6048,6049
0,32,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,70,3.3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,77,0.0,0.131266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,42,3.9,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# final evaluation of models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [22]:
# Random Forest
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)
start = time.time()
rf_model = rf.fit(X_train_vect, y_train)
end = time.time()
fit_time = end - start

pred_start = time.time()
y_pred = rf_model.predict(X_test_vect)
pred_end = time.time()
pred_time = pred_end - pred_start

In [24]:
precision, recall, fscore, support = score(y_test, y_pred, average='binary')
print('Fit time: {} / Prediction Time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                        round(fit_time, 3), round(pred_time, 3),
                                                        round(precision, 3), round(recall, 3), round(y_pred == y_test).sum() / len(y_pred), 3))

Fit time: 5.81 / Prediction Time: 0.147 ---- Precision: 0.984 / Recall: 0.824 / Accuracy: 0.9739910313901345


In [20]:
from joblib import dump
dump(rf_model, 'models/rf({})'.format(round(y_pred == y_test).sum() / len(y_pred), 3))

['models/rf(0.9730941704035875)']

In [26]:
# Gradient Boosting
gb = GradientBoostingClassifier(n_estimators=150, max_depth =11)
fit_start = time.time()
gb_model = gb.fit(X_train_vect, y_train)
fit_end = time.time()
fit_time = fit_end - fit_start

pred_start = time.time()
y_pred = gb_model.predict(X_test_vect)
pred_end = time.time()

pred_time = pred_end - pred_start


In [27]:
precision, recall, fscore, support = score(y_test, y_pred, average='binary')
print('Fit time: {} / Prediction Time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                        round(fit_time, 3), round(pred_time, 3),
                                                        round(precision, 3), round(recall, 3), round(y_pred == y_test).sum() / len(y_pred), 3))

Fit time: 192.007 / Prediction Time: 0.142 ---- Precision: 0.934 / Recall: 0.83 / Accuracy: 0.968609865470852


In [28]:
dump(gb_model, 'models/gb(-recall {})'.format(round(recall, 3)))

['models/gb(-recall 0.83)']